# Main Notebook

File to perform experiments

## Imports

In [1]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf

from models.SOGP import GaussianProcess
from acquisition_functions.SingleObjective import SingleObjectiveAcq

from acquisition_functions.SingleObjective import pi, ei, ucb, mes,  simulated_mes 
from benchmarkFunctions.eggholder import eggholder
from benchmarkFunctions.ackley import ackley


## Algorithm Arguments

In [2]:
seed = 1
np.random.seed(seed)
total_iter = 38
initial_iter = 2

lower_bound = -2
upper_bound = 2


## Evaluation

In [3]:
d = 2

def evaluation(x):
    #return eggholder(np.array([512,x]))
    return np.array([ackley(x)])
    return np.array([1-np.exp(-np.sum(x**2))])

N = 1001
X = np.linspace(lower_bound,upper_bound,N)

if d==1:
    Z = np.zeros(N)
    for i in range(N):
        Z[i]=evaluation(X[i])
    opt_arg, opt_val = Z.argmin(), np.amin(Z)    

elif d==2:
    Z = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            Z[i,j]=evaluation(np.array([X[i],X[j]]))

    opt_arg, opt_val = np.unravel_index(Z.argmin(), Z.shape), np.amin(Z)

In [4]:
def random_acq(GP):
    while True:
        x_rand = np.random.uniform(GP.lowerBound, GP.upperBound, GP.d)
        if GP.X is None or not x_rand in GP.X:
                break
    return x_rand

## N experiments

In [5]:
root_folder = "SingleObjectiveExperiments"
testF = "ackley"
acqF = "simulated_mes5"
function = simulated_mes
df = None
n_experiments = 20

with tf.device('/GPU:1'):

    for i in range(n_experiments):
        print(i, time.ctime())

        ### GPs Initialization
        GP = GaussianProcess(d, lower_bound, upper_bound, noise_variance=2e-6)

        #### Initial samples, at least 1
        for l in range(initial_iter):
            ## Get random evaluation point
            x_rand = random_acq(GP)

            ## EVALUATION OF THE OUTSIDE FUNCTION
            y_rand = evaluation(x_rand)
            GP.addSample(x_rand,y_rand)

        GP.updateGP()
        GP.optimizeKernel()
        if False:
            GP.plotSamples()

        row = {
            'exp_id' : i,
            'testF' : testF,
            'acqF': acqF,
            'time': 0,
            'ns' : len(GP.X),
            'x'  : x_rand,
            'y'  : y_rand,
            'acq': 0
        }
        metrics = GP.evaluateOptimum(opt_val)
        row.update(metrics)
        if df is None:
            df = pd.DataFrame({k: [v] for k, v in row.items()})
        else:
            df = pd.concat([df, pd.DataFrame({k: [v] for k, v in row.items()})])        

        for l in range(total_iter):

            ## Search of the best acquisition function
            start = time.time()
            #x_best, acq = SingleObjectiveAcq(function, GP)
            x_best, acq = simulated_mes(GP, M = 5)
            end = time.time()


            ## EVALUATION OF THE OUTSIDE FUNCTION
            y_best = evaluation(x_best)

            #print(x_best, acq, y_best)
            
            ## UPDATE
            GP.addSample(x_best,y_best)     ## Add new sample to the model
            GP.updateGP()                   ## Update data on the GP regressor
            GP.optimizeKernel()             ## Optimize kernel hyperparameters

            ## Evaluate Pareto (distances and hypervolumes)
            row = {
                'exp_id' : i,
                'testF' : testF,
                'acqF': acqF,
                'time': float(start-end),
                'ns' : len(GP.X),
                'x'  : x_best,
                'y'  : y_best,
                'acq': acq
            }
            metrics = GP.evaluateOptimum(opt_val)
            row.update(metrics)

            df = pd.concat([df, pd.DataFrame({k: [v] for k, v in row.items()})])

0 Thu Jun 16 22:28:55 2022
1 Thu Jun 16 22:29:22 2022
2 Thu Jun 16 22:29:47 2022
3 Thu Jun 16 22:30:11 2022
4 Thu Jun 16 22:30:37 2022
5 Thu Jun 16 22:31:02 2022
6 Thu Jun 16 22:31:26 2022
7 Thu Jun 16 22:31:51 2022
8 Thu Jun 16 22:32:16 2022
9 Thu Jun 16 22:32:42 2022
10 Thu Jun 16 22:33:06 2022
11 Thu Jun 16 22:33:31 2022
12 Thu Jun 16 22:33:56 2022
13 Thu Jun 16 22:34:21 2022
14 Thu Jun 16 22:34:47 2022
15 Thu Jun 16 22:35:12 2022
16 Thu Jun 16 22:35:37 2022
17 Thu Jun 16 22:36:01 2022
18 Thu Jun 16 22:36:27 2022
19 Thu Jun 16 22:36:51 2022


In [6]:
df.to_csv(root_folder+"/"+testF+"/"+acqF+".csv")